In [1]:
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score
import torch
import numpy as np
import pandas as pd
import time

In [2]:
wine_quality = pd.read_csv('./preprocessed-datasets/wine_quality_prepro.csv', index_col=0)
cong_voting = pd.read_csv('./preprocessed-datasets/CongressionVoting_prepro.csv')
# encode class value democrat as 1 and republican as 0
cong_voting['class'] = cong_voting['class'].map({'democrat': 1, 'republican': 0})
bank_marketing = pd.read_csv('./preprocessed-datasets/bank_marketing_prepro.csv')
column_to_move = 'class'

# Move class to the last index
columns = [col for col in bank_marketing.columns if col != column_to_move] + [column_to_move]
bank_marketing = bank_marketing[columns]

bank_marketing.drop('Unnamed: 0', axis=1,inplace=True)

In [3]:
def train_test_split(data: pd.DataFrame, target_label : str, test_size=0.2, return_torch=None):
        
    # split the data into train and test
    #train = data.sample(frac=(1-test_size),random_state=200)
    #test = data.drop(train.index)
    
    # split the train and test into X and Y
    X = data.drop([target_label], axis=1).values
    Y = data[target_label].values
    
    if return_torch:
        train_X = torch.tensor(X)
        train_Y = torch.tensor(Y)
    
    return X, Y

In [4]:
def MLP(X,y, num_splits = 5):
    kf = KFold(n_splits=num_splits, shuffle=True, random_state=42)

    train_accuracies = []
    test_accuracies = []
    train_times = []

    for train_index, test_index in kf.split(X, y):
        
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        
        # MLPClassifier
        mlp_model = MLPClassifier()
        
        # Train MLPClassifier
        t0 = time.time()
        mlp_model.fit(X_train, y_train)
        t1 = time.time()
        training_time_tmp = t1-t0
        # Make predictions on the test set
        mlp_predictions_train = mlp_model.predict(X_train)
        mlp_predictions_test = mlp_model.predict(X_test)

        # Calculate accuracy
        train_accuracy_fold = accuracy_score(y_train, mlp_predictions_train)
        test_accuracy_fold = accuracy_score(y_test, mlp_predictions_test)

        train_accuracies.append(train_accuracy_fold)
        test_accuracies.append(test_accuracy_fold)
        train_times.append(training_time_tmp)
        
    
    return np.mean(train_accuracies), np.mean(test_accuracies), np.mean(train_times)

In [5]:
results = pd.DataFrame(columns=["method",'dataset','Average Accuracy','Average Training Time'])
datasets = {
    'wine_quality': wine_quality,
    'congression_voting': cong_voting,
    'bank_marketing': bank_marketing
}
for dataset_name, dataset in datasets.items():
    X, y = train_test_split(dataset, 'class')    
    _, test_acc, train_time = MLP(X,y)
    results = results.append({'method':'MLP', 'dataset':dataset_name, 'Average Accuracy':test_acc, 'Average Training Time':train_time}, ignore_index=True)

C:\Users\zsomb\AppData\Local\Temp\ipykernel_19656\3915861609.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append({'method':'MLP', 'dataset':dataset_name, 'Average Accuracy':test_acc, 'Average Training Time':train_time}, ignore_index=True)
c:\Users\zsomb\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\zsomb\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\Users\zsomb\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:702: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the op

In [8]:
results

,method,dataset,Average Accuracy,Average Training Time
0,MLP,wine_quality,0.540096,3.460707
1,MLP,congression_voting,0.921776,1.010281
2,MLP,bank_marketing,0.859618,11.061345


In [9]:
results.to_csv('./results/MLP.csv')